<a href="https://colab.research.google.com/github/sazio/first-order-model/blob/master/first_order_model_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "First Order Motion Model for Image Animation"

**Clone repository**

In [2]:
#!git clone https://github.com/AliaksandrSiarohin/first-order-model
!git clone https://github.com/sazio/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 255 (delta 5), reused 0 (delta 0), pack-reused 243
Receiving objects: 100% (255/255), 72.36 MiB | 31.79 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [3]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [16]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os 
warnings.filterwarnings("ignore")


#source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
source_image = imageio.imread("/content/gdrive/My Drive/first-order-motion-model/camma.png")
#driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/04.mp4')
driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/plomo.mp4')

#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [6]:
!pip install moviepy

In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                          checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')#, cpu = True)

Audio Extraction with MoviePy


In [8]:
path_to_vid = '/content/gdrive/My Drive/first-order-motion-model/plomo.mp4'
path_to_audio = path_to_vid[:50] +  path_to_vid[50:-4] + '-audio'+ '.wav'

In [12]:
import moviepy.editor
video_mpy = moviepy.editor.VideoFileClip(path_to_vid)
audio_mpy = video_mpy.audio

audio_mpy.write_audiofile(path_to_audio)

[MoviePy] Writing audio in /content/gdrive/My Drive/first-order-motion-model/plomo-audio.wav


100%|██████████| 90/90 [00:00<00:00, 200.21it/s]

[MoviePy] Done.


**Perform image animation**

In [18]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)


#save resulting video
path_gen_vid = '/content/gdrive/My Drive/first-order-motion-model/generated.mp4'
imageio.mimsave(path_gen_vid, [img_as_ubyte(frame) for frame in predictions], fps = 30)
#video can be downloaded from /content folder

#HTML(display(source_image, driving_video, predictions).to_html5_video())
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 122/122 [00:05<00:00, 24.26it/s]


In [17]:
path_final_vid = '/content/gdrive/My Drive/first-order-motion-model/final_vid.mp4'
os.environ["inputVideo"] = path_gen_vid
os.environ["inputAudio"] = path_to_audio
os.environ["outputVideo"] = path_final_vid

!ffmpeg -hide_banner -i "$inputVideo" -i "$inputAudio" -c:v copy  -c:a aac "$outputVideo"

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/content/gdrive/My Drive/first-order-motion-model/generated.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.72.101
  Duration: 00:00:04.07, start: 0.000000, bitrate: 116 kb/s
    Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p, 256x256, 111 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      handler_name    : VideoHandler
Guessed Channel Layout for Input Stream #1.0 : stereo
Input #1, wav, from '/content/gdrive/My Drive/first-order-motion-model/plomo-audio.wav':
  Metadata:
    encoder         : Lavf57.72.101
  Duration: 00:00:04.07, bitrate: 1411 kb/s
    Stream #1:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, stereo, s16, 1411 kb/s
File '/content/gdrive/My Drive/first-order-motion-model/final_vid.mp4' already exists. Overwrite ? [y/N] y
Stream mapping:
  Stream #0:0 -> #0:0 (copy)
  Stream #1:0 -> #0:1 (pcm_s